In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.layers import Dense, GRU, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [35]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    texts_true = f.readlines()
    texts_true[0] = texts_true[0].replace('\ufeff', '') #убираем первый невидимый символ

with open('train_data_false.txt', 'r', encoding='utf-8') as f:
    texts_false = f.readlines()
    texts_false[0] = texts_false[0].replace('\ufeff', '') #убираем первый невидимый символ

texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

84 88 172


In [36]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

[('думайте', 1), ('позитивно', 4), ('и', 50), ('верьте', 3), ('в', 38), ('свою', 4), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1)]
Думайте позитивно и верьте в свою способность достигать отличных результатов. 



In [37]:
max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

[[197  54   2 ... 199 200 201]
 [  0   4 202 ... 205   3  67]
 [206   3  67 ...   4 208 209]
 ...
 [  0  20  62 ...  53 850 851]
 [  0   0  43 ...  33   1 853]
 [  0   0   0 ...  70  65 194]]


In [38]:
#print( list(tokenizer.word_index.items()) )

In [45]:
X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

(1, 10) (172, 2)


In [58]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(GRU(128, return_sequences=True)) # return_sequences - чтобы выходные знач-я соответствовали входным значениям следующего слоя
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 10, 128)           128000    
                                                                 
 gru_18 (GRU)                (None, 10, 128)           99072     
                                                                 
 gru_19 (GRU)                (None, 64)                37248     
                                                                 
 dense_9 (Dense)             (None, 2)                 130       
                                                                 
Total params: 264,450
Trainable params: 264,450
Non-trainable params: 0
_________________________________________________________________


In [59]:
his = model.fit(X, Y, batch_size=32, epochs=50)

#plt.plot(his.history['loss']) # 2й history - словарь, loss - критерий качества
#, вычисленный для каждой эпохи
#plt.plot(his.history['val_loss'])
#plt.grid(True)
#plt.show()

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 0.7006 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6925 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6844 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6763 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6683 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6604 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6525 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6445 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6367 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6288 - accuracy: 1.0000
Epoch 11/50
1/1 [==========

In [60]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

In [61]:
t = "Мне никогда не выздороветь".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')

['мне', 'никогда', 'не', 'выздороветь']
[[0.7097477 0.2902523]]
0
